上一章介绍了多层感知机在内的简单深度学习模型的原因和实现。本章将简要概括深度学习计算的各个重要组成部分，如模型构造、参数的访问和初始化等，自定义层，读取、存储和使用GPU。通过本章的学习，我们将能够深入了解模型实现和计算的各个细节，并为在之后章节实现更复杂模型打下坚实的基础。

# 4.1 模型构造

让我们回顾一下在[“多层感知机的简洁实现”](../chapter03_deep-learning-basics/3.10_mlp-pytorch.ipynb)一节中含单隐藏层的多层感知机的实现方法。我们首先构造`Sequential`实例，然后依次添加两个全连接层。其中第一层的输出大小为256，即隐藏层单元个数是256；第二层的输出大小为10，即输出层单元个数是10。我们在上一章的其他节中也使用了`Sequential`类构造模型。这里将详细全面地介绍另外一种基于`Module`类的模型构造方法：它让模型构造更加灵活。

## 4.1.1 继承`Module`类来构造模型

[`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#module)类是[`torch.nn`](https://pytorch.org/docs/stable/nn.html#module-torch.nn)模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。下面继承`Module`类构造本节开头提到的多层感知机。这里定义的`MLP`类重载了`Module`类的`__init__`函数和`forward`函数。它们分别用于创建模型参数和定义前向计算。前向计算也即正向传播。

In [1]:
import torch
from torch import nn

print(torch.__version__)

1.2.0


In [5]:
class MLP(nn.Module):
    # 声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        # 参数，如“模型参数的访问、初始化和共享”一节将介绍的模型参数params
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256) # 隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)  # 输出层
         

    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

以上的`MLP`类中无须定义反向传播函数。系统将通过自动求梯度而自动生成反向传播所需的`backward`函数。

我们可以实例化`MLP`类得到模型变量`net`。下面的代码初始化`net`并传入输入数据`X`做一次前向计算。其中，`net(X)`会调用`MLP`继承自`Module`类的`__call__`函数，这个函数将调用`MLP`类定义的`forward`函数来完成前向计算。

In [6]:
X = torch.rand(2, 784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1432, -0.1739, -0.2414,  0.0252,  0.2747, -0.1585,  0.1063, -0.0813,
         -0.1491, -0.2109],
        [ 0.2423, -0.1115, -0.1969, -0.0829,  0.2976,  0.0086, -0.0342, -0.0612,
         -0.2168, -0.2097]], grad_fn=<AddmmBackward>)

注意，这里并没有将`Module`类命名为`Layer`（层）或者`Model`（模型）之类的名字，这是因为该类是一个可供自由组建的部件。它的子类既可以是一个层（如PyTorch提供的`Linear`类），又可以是一个模型（如这里定义的`MLP`类），或者是模型的一个部分。下面通过两个例子来展示它的灵活性。

## 4.1.2 `Sequential`类继承自`Module`类

我们刚刚提到，[`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)类是一个通用的部件。事实上，`torch.nn.Sequential`类继承自`torch.nn.Module`类。当模型的前向计算为简单串联各个层的计算时，可以通过更加简单的方式定义模型。这正是`Sequential`类的目的：它提供`add`函数来逐一添加串联的`Module`子类实例，而模型的前向计算就是将这些实例按添加的顺序逐一计算。

下面我们实现一个与`Sequential`类有相同功能的`MySequential`类。这或许可以帮助读者更加清晰地理解`Sequential`类的工作机制。

In [7]:
class MySequential(nn.Module):
    
    from collections import OrderedDict 
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) == 1 and isinstance(args[0], OrderedDict): # 如果传入的是一个OrderedDict
            for key, module in args[0].items():
                self.add_module(key, module)  # add_module方法会将module添加进self._modules(一个OrderedDict)
        else:  # 传入的是一些Module
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    
    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成
        for module in self._modules.values():
            input = module(input)
        return input

我们用`MySequential`类来实现前面描述的`MLP`类，并使用随机初始化的模型做一次前向计算。

In [8]:
net = MySequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10), 
        )
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1525,  0.0448,  0.1585,  0.1523, -0.0428,  0.2494,  0.0754, -0.0729,
          0.0270,  0.1745],
        [-0.1905, -0.0969,  0.0515,  0.1284,  0.0945,  0.2423, -0.0553,  0.0029,
         -0.1191,  0.0773]], grad_fn=<AddmmBackward>)

可以观察到这里`MySequential`类的使用跟3.10节“[多层感知机的简洁实现](../chapter03_deep-learning-basics/3.10_mlp-pytorch.ipynb)”中`Sequential`类的使用没什么区别。

###  `ModuleList`类

[`torch.nn.ModuleList`](https://pytorch.org/docs/stable/nn.html#torch.nn.ModuleList)接收一个子模块的列表作为输入，然后也可以类似List那样进行append和extend操作:

In [11]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10)) # # 类似List的append操作
print(net[-1])  # 类似List的索引访问，即最后一个列表节点
print(net)
print(net[1])

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
ReLU()


###  `ModuleDict`类

[`torch.nn.ModuleDict`](https://pytorch.org/docs/stable/nn.html#moduledict)接收一个子模块的字典作为输入, 然后也可以类似字典那样进行添加访问操作:☟

In [7]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) # 添加
print(net['linear']) # 访问
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


除了上述三个类之外，PyTorch还提供了[ParameterList](https://pytorch.org/docs/stable/nn.html#parameterlist)类和[ParameterDict](https://pytorch.org/docs/stable/nn.html#parameterdict)类，详细可以点击链接精读，在此不再赘述。

## 4.1.3 构造复杂的模型

虽然上面介绍的这些类可以使模型构造更加简单，且不需要定义`forward`函数，但直接继承`Module`类可以极大地拓展模型构造的灵活性。下面我们构造一个稍微复杂点的网络`FancyMLP`。在这个网络中，我们通过`get_constant`函数创建训练中不被迭代的参数，即常数参数。在前向计算中，除了使用创建的常数参数外，我们还使用`Tensor`的函数和Python的控制流，并多次调用相同的层。

In [13]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        
        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

在这个`FancyMLP`模型中，我们使用了常数权重`rand_weight`（注意它不是可训练模型参数）、做了矩阵乘法操作（`torch.mm`）并重复使用了相同的`Linear`层。下面我们来测试该模型的前向计算。

In [14]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-3.1365, grad_fn=<SumBackward0>)

因为`FancyMLP`和`Sequential`类都是`Module`类的子类，所以我们可以嵌套调用它们。

In [15]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        #self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU()) 
        self.net = nn.Sequential()
        self.net.add_module('0', nn.Linear(64, 10))

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(0.7849, grad_fn=<SumBackward0>)

## 小结

* 可以通过继承`Module`类来构造模型。
* `Sequential`类继承自`Module`类。
* 虽然`Sequential`类可以使模型构造更加简单，但直接继承`Module`类可以极大地拓展模型构造的灵活性。

## 练习

* 如果不在`MLP`类的`__init__`函数里调用父类的`__init__`函数，会出现什么样的错误信息？
* 如果去掉`FancyMLP`类里面的`item()`函数，会有什么问题？
* 如果将`NestMLP`类中通过`Sequential`实例定义的`self.net`改为`self.net = [nn.Dense(64, activation='relu'), nn.Dense(32, activation='relu')]`，会有什么问题？